In [56]:
import re
import uuid
from dataclasses import dataclass, field
from typing import List, Dict, Any, Tuple
from pathlib import Path

# --------- 유틸: 키 정규화 / 용어 추출 ---------
_WORD_SPLIT_RE = re.compile(r"[^\w]+", re.UNICODE)
CAMEL_RE = re.compile(r"(?<!^)(?=[A-Z])")

def normalize_key(s: str) -> str:
    # 대소문자/구분자 정규화 (camel/snake/kebab → 토큰화 후 소문자)
    s = s.strip()
    s = CAMEL_RE.sub(" ", s)
    toks = _WORD_SPLIT_RE.split(s.lower())
    toks = [t for t in toks if t]
    return " ".join(toks)

def extract_exact_keys(text: str) -> List[str]:
    keys = []
    # 코드값/상수/식별자 패턴들 (원문 보존)
    patterns = [
        r"\b[A-Z0-9_]{3,}\b",                 # E401, SINGLE_PAYMENT_TRANSACTION, MKT_ONE
        r"\b[A-Za-z_]+=[A-Za-z0-9._-]+\b",    # KEY=VALUE
        r'"[A-Za-z0-9_.-]+"\s*:',             # "msgVersion":  (JSON 필드명)
    ]
    for pat in patterns:
        for m in re.finditer(pat, text):
            keys.append(m.group(0).strip('"').rstrip(":"))
    # 중복 제거, 과도한 수는 상한
    dedup = []
    seen = set()
    for k in keys:
        if k not in seen:
            seen.add(k); dedup.append(k)
    return dedup[:200]

def extract_term_keys(title: str, body: str) -> List[str]:
    candidates = []
    # 제목/굵은글/정의형 문장 등에서 후보 수집(간단 규칙)
    candidates.append(title)
    for m in re.finditer(r"\*\*(.+?)\*\*", body):
        candidates.append(m.group(1))
    for m in re.finditer(r"(.+?)(?:란|은|는)\s.+?(?:이다|입니다)\.", body):
        if len(m.group(1)) <= 40:
            candidates.append(m.group(1))
    # 정규화 + dedup
    norm = []
    seen = set()
    for c in candidates:
        k = normalize_key(c)
        if k and k not in seen:
            seen.add(k); norm.append(k)
    return norm[:100]

# --------- 마크다운 파서(간단): H1~H3 섹션/코드/표/이미지 ---------
CODE_BLOCK_RE = re.compile(r"```([a-zA-Z0-9_-]*)[^\n]*\n(.*?)```", re.DOTALL)
TABLE_RE = re.compile(r"(?:^\|.*\|\s*\n)+", re.MULTILINE)  # Markdown 표 블록 대략 매칭
IMAGE_RE = re.compile(r"!\[(.*?)\]\((.*?)(?:\s+\"(.*?)\")?\)")

@dataclass
class Tile:
    id: str
    type: str  # definition|explanation|code|table|image
    text: str
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass
class SectionPack:
    id: str
    title: str
    heading_path: str
    text: str               # 섹션 전체 텍스트(코드/표 캡션 요약 포함)
    tiles: List[Tile]
    metadata: Dict[str, Any] = field(default_factory=dict)

def split_markdown_to_sections_hierarchical(md_text: str, doc_id: str) -> List[SectionPack]:
    """
    계층구조를 보존하는 마크다운 분할 함수
    
    H2/H3 기준 섹션을 만들고, 섹션별로 타일(정의/설명/코드/표/이미지)을 생성.
    코드/표는 '원자' 보존: 절대로 분할하지 않음.
    계층구조를 보존하여 title에 전체 경로를 포함합니다.
    """
    # 헤더 파싱: 모든 헤더(H1~H6) 라인 인덱스 수집
    lines = md_text.splitlines()
    header_idxs: List[Tuple[int, str, int]] = []  # (line_idx, title, level)
    for i, ln in enumerate(lines):
        m = re.match(r"^(#{1,6})\s+(.*)", ln)  # H1~H6 모두 포함
        if m:
            level = len(m.group(1))
            header_idxs.append((i, m.group(2).strip(), level))

    # 계층구조를 보존하여 경로 생성
    current_hierarchy: List[str] = []  # 현재까지의 계층 구조
    hierarchical_headers: List[Tuple[int, str, int, str]] = []  # (line_idx, title, level, full_path)
    
    for line_idx, title, level in header_idxs:
        # 계층 구조 업데이트
        if level == 1:
            current_hierarchy = [title]
        elif level <= len(current_hierarchy):
            current_hierarchy = current_hierarchy[:level-1] + [title]
        else:
            current_hierarchy.append(title)
        
        # 전체 경로 생성
        full_path = ' > '.join(current_hierarchy)
        hierarchical_headers.append((line_idx, title, level, full_path))

    # H2/H3 기준으로 섹션 경계 설정 (계층구조 정보 포함)
    h2h3_headers = [(idx, title, level, path) for idx, title, level, path in hierarchical_headers 
                    if level in [2, 3]]
    
    boundaries = []
    for idx, (line_idx, title, level, full_path) in enumerate(h2h3_headers):
        start = line_idx
        end = h2h3_headers[idx+1][0] if idx+1 < len(h2h3_headers) else len(lines)
        boundaries.append((start, end, title, level, full_path))

    section_packs: List[SectionPack] = []

    for start, end, title, level, full_path in boundaries:
        raw = "\n".join(lines[start:end])
        section_id = str(uuid.uuid4())

        # 코드블록 추출(원문 보존)
        code_tiles = []
        def code_repl(m):
            lang = (m.group(1) or "").lower()
            code = m.group(2)
            tile = Tile(
                id=str(uuid.uuid4()),
                type="code",
                text=code.strip(),
                metadata={"code_lang": lang or "text"}
            )
            code_tiles.append(tile)
            return f"\n[CODE_BLOCK::{tile.id}]\n"  # 자리표시자

        raw_wo_code = re.sub(CODE_BLOCK_RE, code_repl, raw, flags=re.DOTALL)

        # 표 추출(원문 보존)
        table_tiles = []
        def table_repl(m):
            tbl = m.group(0)
            tile = Tile(
                id=str(uuid.uuid4()),
                type="table",
                text=tbl.strip(),
                metadata={}
            )
            table_tiles.append(tile)
            return f"\n[TABLE_BLOCK::{tile.id}]\n"

        raw_wo_code_table = re.sub(TABLE_RE, table_repl, raw_wo_code, flags=re.MULTILINE)

        # 이미지 캡션 추출
        image_tiles = []
        def image_repl(m):
            alt, path, title_opt = m.group(1), m.group(2), m.group(3) or ""
            caption = (alt or title_opt or path)
            tile = Tile(
                id=str(uuid.uuid4()),
                type="image",
                text=caption.strip(),
                metadata={"image_path": path}
            )
            image_tiles.append(tile)
            return f"\n[IMAGE_BLOCK::{tile.id}]\n"

        raw_clean = re.sub(IMAGE_RE, image_repl, raw_wo_code_table)

        # 정의/설명 타일 만들기
        # 첫 단락을 definition, 나머지를 explanation 타일들로 (문단 기준 분할, 10~15% 겹침 없음)
        paragraphs = [p.strip() for p in re.split(r"\n\s*\n", raw_clean) if p.strip()]
        tiles: List[Tile] = []

        if paragraphs:
            tiles.append(Tile(
                id=str(uuid.uuid4()),
                type="definition",
                text=paragraphs[0],
                metadata={}
            ))
            for p in paragraphs[1:]:
                tiles.append(Tile(
                    id=str(uuid.uuid4()),
                    type="explanation",
                    text=p,
                    metadata={}
                ))

        # 코드/표/이미지 타일 추가 (원자 보존)
        tiles.extend(code_tiles)
        tiles.extend(table_tiles)
        tiles.extend(image_tiles)

        # 메타데이터 구성
        heading_path = full_path  # 계층구조 전체 경로 사용
        body_for_keys = "\n".join(t.text for t in tiles if t.type in ("definition","explanation"))
        term_keys = extract_term_keys(title, body_for_keys)
        exact_keys = extract_exact_keys(raw)

        # 섹션팩 텍스트(LLM 주입용): 자리표시자 → 간단 요약문으로 치환
        def restore_placeholders(s: str) -> str:
            s = re.sub(r"\[CODE_BLOCK::([-\w]+)\]", "[코드블록: 전체 포함]", s)
            s = re.sub(r"\[TABLE_BLOCK::([-\w]+)\]", "[표: 전체 포함]", s)
            s = re.sub(r"\[IMAGE_BLOCK::([-\w]+)\]", "[이미지 캡션 포함]", s)
            return s

        section_text = restore_placeholders(raw_clean)

        pack = SectionPack(
            id=section_id,
            title=full_path,  # 계층구조 전체 경로를 title로 사용
            heading_path=heading_path,
            text=section_text.strip(),
            tiles=tiles,
            metadata={
                "doc_id": doc_id,
                "section_title": title,  # 원래 제목
                "heading_path": heading_path,  # 전체 경로
                "term_keys": term_keys,
                "exact_keys": exact_keys,
            }
        )
        # 타일 공통 메타 바인딩
        for t in pack.tiles:
            t.metadata.update({
                "doc_id": doc_id,
                "section_id": section_id,
                "section_title": title,  # 원래 제목
                "heading_path": heading_path,  # 전체 경로
            })
            if t.type == "table":
                # 표 헤더/키 후보 추출(간단): 1행을 헤더로 가정
                lines_tbl = [ln for ln in t.text.splitlines() if ln.strip()]
                if lines_tbl:
                    t.metadata["table_header"] = lines_tbl[0]
            if t.type == "code":
                t.metadata["exact_keys"] = extract_exact_keys(t.text)

        section_packs.append(pack)

    return section_packs


In [57]:
### TEST_CODE ###

def read_file_content(input_path: str) -> str:
    try:
        # UTF-8 인코딩으로 파일 읽기
        with open(input_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {input_path}")
          
all_text = read_file_content("data/dev_center_guide_allmd_touched.md") 
# print(all_text)

list_section = split_markdown_to_sections_hierarchical(all_text, doc_id="iap_dev_integration_guide")

cnt = 0
for section in list_section:
    print(section)
    cnt += 1
    # if 'CANCELED' in section.text:
    #     print(section.title)
    #     print("=" * 50)
    #     print(section.text)
    #     print("-" * 100)
    #     cnt += 1

print(f"문장 개수: {cnt}")

ValueError: cannot process flags argument with a compiled pattern

In [43]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document
from langchain.retrievers import EnsembleRetriever
from langchain.storage import InMemoryStore

def build_retrievers_from_md(md_path: str, doc_id: str):
    md = Path(md_path).read_text(encoding="utf-8")
    packs = split_markdown_to_sections(md, doc_id=doc_id)

    child_docs: List[Document] = []
    for pack in packs:
        for t in pack.tiles:
            meta = dict(t.metadata)
            term_keys = " ".join(pack.metadata.get("term_keys", []))
            exact_keys = " ".join(pack.metadata.get("exact_keys", []))
            if t.type == "code":
                exact_keys += " " + " ".join(t.metadata.get("exact_keys", []))
            meta["term_keys"] = term_keys
            meta["exact_keys"] = exact_keys
            meta["tile_type"] = t.type
            child_docs.append(Document(page_content=t.text, metadata=meta))

    # --- 임베딩: Ollama bge-m3 ---
    emb = OllamaEmbeddings(model="bge-m3:latest")  # GPU/CPU 자동 판단

    child_vectorstore = FAISS.from_documents(
        child_docs, emb, distance_strategy=DistanceStrategy.COSINE
    )

    # --- BM25 (정확일치 강화: term_keys/exact_keys 포함) ---
    bm25_docs = []
    for d in child_docs:
        extra = f"\n{d.metadata.get('term_keys','')}\n{d.metadata.get('exact_keys','')}"
        bm25_docs.append(Document(page_content=d.page_content + extra, metadata=d.metadata))
    bm25 = BM25Retriever.from_documents(bm25_docs); bm25.k = 12

    vector_retriever = child_vectorstore.as_retriever(search_kwargs={"k": 12})
    ensemble = EnsembleRetriever(retrievers=[bm25, vector_retriever], weights=[0.55, 0.45])

    # --- Parent Join 준비 ---
    parent_store = InMemoryStore()
    parent_docs = []
    for p in packs:
        parent_docs.append(Document(
            page_content=p.text,
            metadata=p.metadata | {"section_id": p.id, "section_title": p.title}
        ))
    for d in parent_docs:
        parent_store.mset([(d.metadata["section_id"], d)])

    child_store = InMemoryStore()
    for d in child_docs:
        child_store.mset([(d.metadata["section_id"], d)])
        # child_store.madd([(d.metadata["section_id"], d)])

    return packs, ensemble, parent_store, child_vectorstore

def hybrid_parent_search(query: str, ensemble: EnsembleRetriever, parent_store: InMemoryStore, k: int = 4) -> List[Document]:
    child_hits = ensemble.get_relevant_documents(query)
    bucket_by_section = {}
    for ch in child_hits:
        sid = ch.metadata.get("section_id")
        bucket_by_section.setdefault(sid, []).append(ch)
    ordered_sections = list(bucket_by_section.keys())[:k]
    parent_results = []
    for sid in ordered_sections:
        doc = parent_store.mget([sid])[0]
        parent_results.append(doc)
    return parent_results

In [45]:
from langchain_community.chat_models import ChatOllama
from langchain.schema import SystemMessage, HumanMessage

def build_llm():
    # 필요 시 num_ctx(컨텍스트 길이)나 num_predict(출력 길이) 조정 가능
    llm = ChatOllama(
        model="exaone3.5:latest",
        temperature=0,
        # num_ctx=8192,   # 모델/로컬 자원에 맞게 조절
        # num_predict=1024
    )
    return llm

def answer(query: str, ensemble: EnsembleRetriever, parent_store: InMemoryStore, llm: ChatOllama, k: int = 4) -> str:
    parents = hybrid_parent_search(query, ensemble, parent_store, k=k)
    context_blocks = []
    for i, p in enumerate(parents, 1):
        context_blocks.append(f"[Section {i}: {p.metadata.get('heading_path')}]\\n{p.page_content}")

    system = SystemMessage(content=(
        "당신은 기술 문서 기반 QA 어시스턴트입니다. "
        "반드시 제공된 컨텍스트만으로 답하고, 불확실하면 명시하세요."
    ))
    user = HumanMessage(content=(
        f"질문: {query}\n\n컨텍스트:\n" + "\n\n".join(context_blocks)
    ))
    res = llm([system, user])
    return res.content

# =============== 실행 예시 ===============
# if __name__ == "__main__":
#     # 샘플 마크다운 파일 경로
#     md_path = "pns_sample.md"   # 제공하신 PNS 예제를 저장한 파일
#     packs, ensemble, parent_store, child_vectorstore = build_retrievers_from_md(md_path, doc_id="pns_doc_v1")
#     llm = build_llm()

#     q1 = "PNS 3.1.0에서 packageName이 clientId로 바뀐 이유와 주의사항을 알려줘"
#     q2 = "paymentMethod의 종류와 설명 전체를 보여줘"
#     q3 = "messageType이 SINGLE_PAYMENT_TRANSACTION일 때 JSON 필드 목록"

#     print("Q1:", answer(q1, ensemble, parent_store, llm))
#     print("Q2:", answer(q2, ensemble, parent_store, llm))
#     print("Q3:", answer(q3, ensemble, parent_store, llm))

In [47]:

# 샘플 마크다운 파일 경로
md_path = "data/dev_center_guide_allmd_touched.md"   # 제공하신 PNS 예제를 저장한 파일
packs, ensemble, parent_store, child_vectorstore = build_retrievers_from_md(md_path, doc_id="pns_doc_v1")
llm = build_llm()

q1 = "PNS는 무엇입니까?"
q2 = "PNS의 메세지의 purchaseState는 어떤 값들이 있습니까?"
q3 = "Push Notification과 Payment Notification의 차이는 무엇입니까?"

print("Q1", answer(q1, ensemble, parent_store, llm))
print("Q2:", answer(q2, ensemble, parent_store, llm))
print("Q3:", answer(q3, ensemble, parent_store, llm))

Q1 PNS는 **Payment Notification Service**의 약자로, 모바일 네트워크 연결의 불안정성을 보완하기 위해 개발사가 지정한 서버로 원스토어가 사용자의 결제 상태 (결제 완료, 결제 취소 등) 정보를 메시지 형태로 전송하는 기능입니다. 이를 통해 결제 트랜잭션의 상태를 안정적으로 전달하고 관리할 수 있습니다.
Q2: 제공된 컨텍스트에서 PNS (아마도 원스토어 인앱 결제 시스템을 가리키는 것으로 추정됨)의 `purchaseState`에 대한 구체적인 값 목록은 명시적으로 언급되어 있지 않습니다. 주어진 정보는 주로 인앱 상품의 유형 (관리형 상품, 구독형 상품 등)과 관련된 파라미터 및 메서드에 초점을 맞추고 있습니다.

`purchaseState`에 대한 일반적인 인앱 결제 시스템의 값들은 다음과 같을 수 있지만, 이는 특정 컨텍스트나 문서에 따라 다를 수 있습니다:

- **PENDING**: 구매 요청이 아직 처리 중인 상태
- **COMPLETED**: 구매가 성공적으로 완료된 상태
- **REFUNDED**: 환불이 이루어진 상태
- **FAILED**: 구매 과정에서 오류가 발생하여 실패한 상태

하지만, 이 값들은 제공된 컨텍스트에서 직접적으로 확인된 것이 아니므로, 정확한 `purchaseState` 값들을 알고 싶으시다면 원스토어 인앱 결제 API 문서나 관련 SDK 문서를 직접 참조하시는 것이 가장 정확할 것입니다. 만약 추가 정보가 필요하시다면, 해당 문서나 공식 지원 채널을 통해 확인하시는 것을 권장드립니다.
Q3: Push Notification과 Payment Notification의 주요 차이점은 다음과 같습니다:

1. **목적**:
   - **Push Notification**: 주로 사용자에게 실시간으로 정보를 전달하는 데 사용됩니다. 이는 앱 내에서 사용자에게 새로운 업데이트, 메시지, 이벤트 알림 등을 즉시 알리는 데 초점을 맞춥니다. 예를 들어, 새로운 콘텐츠 알림, 친구 요청, 중요한 앱 이벤트 